# 類似画像検索

## パスの設定
`image_path_s3` に、検索対象となる画像が格納されている S3 のパスを記載します。

In [ ]:
image_path_s3 = 's3://bucket/directory/'

## 実行環境の設定

In [ ]:
!pip install hnswlib
!pip install gluoncv

In [ ]:
import mxnet as mx
from mxnet import gluon, nd
from mxnet.gluon.model_zoo import vision
import multiprocessing
from mxnet.gluon.data.vision.datasets import ImageFolderDataset
from mxnet.gluon.data import DataLoader
import numpy as np
# import wget
import imghdr
import json
import pickle
import hnswlib
import numpy as np
import glob, os, time
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
import urllib.parse
import urllib
import gzip
import os
import tempfile
import glob
from os.path import join
%matplotlib inline

## 機械学習モデルの設定
このサンプルでは、画像から特徴ベクトルに変換するために学習済みの機械学習モデルを使用します。<br>
ここでは、MXNet の model-zoo のモデルを使用します。model-zoo のネットワークは、特徴量が .features プロパティにあり、出力が .output プロパティにあります。この仕組みを利用して、事前にトレーニングされたネットワークを使って featurizer を非常に簡単に作成できます。

In [ ]:
BATCH_SIZE = 256
EMBEDDING_SIZE = 512
SIZE = (224, 224)
MEAN_IMAGE= mx.nd.array([0.485, 0.456, 0.406])
STD_IMAGE = mx.nd.array([0.229, 0.224, 0.225])

In [ ]:
ctx = mx.gpu() if len(mx.test_utils.list_gpus()) else mx.cpu()

In [ ]:
net = vision.resnet18_v2(pretrained=True, ctx=ctx).features

In [ ]:
net.hybridize()

In [ ]:
def transform(image, label):
    resized = mx.image.resize_short(image, SIZE[0]).astype('float32')
    cropped, crop_info = mx.image.center_crop(resized, SIZE)
    cropped /= 255.
    normalized = mx.image.color_normalize(cropped,
                                      mean=MEAN_IMAGE,
                                      std=STD_IMAGE) 
    transposed = nd.transpose(normalized, (2,0,1))
    return transposed, label

## データの準備
S3 から画像をダウンロードします。

In [ ]:
image_path = './cats'

In [ ]:
empty_folder = tempfile.mkdtemp()
# Create an empty image Folder Data Set
dataset = ImageFolderDataset(root=empty_folder, transform=transform)

In [ ]:
!aws s3 cp $image_path_s3 $image_path --recursive

In [ ]:
list_files = glob.glob(os.path.join(image_path, '**.jpg'))

In [ ]:
print("[{}] images".format(len(list_files)))

In [ ]:
dataset.items = list(zip(list_files, [0]*len(list_files)))

In [ ]:
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, last_batch='keep', shuffle=False, num_workers=multiprocessing.cpu_count())

## 画像から特徴ベクトルに変換
機械学習モデルを使って画像を特徴ベクトルに変換します。

In [ ]:
features = np.zeros((len(dataset), EMBEDDING_SIZE), dtype=np.float32)

In [ ]:
%%time
tick = time.time()
n_print = 100
j = 0
for i, (data, label) in enumerate(dataloader):
    data = data.as_in_context(ctx)
    if i%n_print == 0 and i > 0:
        print("{0} batches, {1} images, {2:.3f} img/sec".format(i, i*BATCH_SIZE, BATCH_SIZE*n_print/(time.time()-tick)))
        tick = time.time()
    output = net(data)
    features[(i)*BATCH_SIZE:(i+1)*max(BATCH_SIZE, len(output)), :] = output.asnumpy().squeeze()

## 検索のための準備
このサンプルでは、hnswlib を使って類似ベクトルを検索します。<br>
ここでは、hnswlib のセットアップをします。

In [ ]:
# Number of elements in the index
num_elements = len(features)
labels_index = np.arange(num_elements)

In [ ]:
# Declaring index
p = hnswlib.Index(space = 'cosine', dim = EMBEDDING_SIZE) # possible options are l2, cosine or ip

In [ ]:
%%time 
# Initing index - the maximum number of elements should be known beforehand
p.init_index(max_elements = num_elements, ef_construction = 100, M = 16)

# Element insertion (can be called several times):
int_labels = p.add_items(features, labels_index)


[efパラメーター](https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md) で定義された、クエリ時間の精度と速度のトレードオフを設定します。
ここでは、最近傍の動的リストのサイズを設定しています（検索中に使用されます）。 設定した値が大きいほど、検索はより正確ですが遅くなります。 この設定値は、クエリされた最近傍の数kより小さな値を設定することはできません。この設定値は、kとデータセットのサイズの間の任意の値を設定可能です。

現在、パラメータはインデックスと一緒に保存されないため、ロード後に手動で設定する必要があることに注意してください。

In [ ]:
# Controlling the recall by setting ef:
p.set_ef(300) # ef should always be > k

In [ ]:
p.save_index(join('mms', 'index.idx'))

In [ ]:
p.load_index(join('mms','index.idx'))

## 類似画像の検索

In [ ]:
def plot_predictions(images):
    rows = len(images)//3+2
    gs = gridspec.GridSpec(rows, 3)
    fig = plt.figure(figsize=(15, 5*rows))
    gs.update(hspace=0.1, wspace=0.1)
    for i, (gg, image) in enumerate(zip(gs, images)):
        gg2 = gridspec.GridSpecFromSubplotSpec(10, 10, subplot_spec=gg)
        ax = fig.add_subplot(gg2[:,:])
        ax.imshow(image, cmap='Greys_r')
        ax.tick_params(axis='both',       
                       which='both',      
                       bottom='off',      
                       top='off',         
                       left='off',
                       right='off',
                       labelleft='off',
                       labelbottom='off') 
        ax.axes.set_title("result [{}]".format(i))
        if i == 0:
            plt.setp(ax.spines.values(), color='red')
            ax.axes.set_title("SEARCH".format(i))

In [ ]:
import time

def search(N, k):
    # Query dataset, k - number of closest elements (returns 2 numpy arrays)
    start = time.time()
    q_labels, q_distances = p.knn_query([features[N]], k = k+1)
    time_for_query = (time.time()- start)*1000
    print('time for query: ', str(time_for_query)+' msec')
    images = [plt.imread(dataset.items[label][0]) for label in q_labels[0][1:]]
    plot_predictions(images)

用意した画像の中からランダムに 1枚の画像を選び、その画像と類似する画像を検索して表示します。

In [ ]:
%%time
index = np.random.randint(0,len(features))
print(index)
k = 6
search(index, k)